In [1]:
import pandas as pd
import numpy as np


In [2]:
data = pd.read_csv("Assignment_2_Data_and_Template.csv")

In [3]:
data.head()

,Sex,Height,HandSpan
0,Female,68,21.5
1,Male,71,23.5
2,Male,73,22.5
3,Female,64,18.0
4,Male,68,23.5


In [4]:
def histClassifier(X,T, Bx ,By ,xmin,xmax):
    HF=np.zeros((Bx,By)).astype('int32')
    HM = np.zeros((Bx,By)).astype('int32')
    binindexHeight = (np.round((Bx-1)*(X['Height'] - xmin['Height'])/(xmax['Height'] - xmin['Height']))).astype('int32')
    binindexHandSpan = (np.round((By-1)*(X['HandSpan'] - xmin['HandSpan'])/(xmax['HandSpan'] - xmin['HandSpan']))).astype('int32')
    smallIndex = min(len(binindexHeight),len(binindexHandSpan))
    print("Small Index" , len(binindexHandSpan))
    for (i,b) in enumerate(binindexHeight):
        if T[i] == 'Female':
            HF[b][binindexHandSpan[i]]+=1
        else:
            HM[b][binindexHandSpan[i]]+=1
    return [HF,HM]



In [5]:
data['Sex'].count()

167

In [6]:
import math
N = data['Sex'].count()
Bx=math.ceil( math.log2(N+ 1))
By=math.ceil( math.log2(N+ 1))
print("B - ", Bx)

B -  8


In [7]:
xmin = data.min()
xmax = data.max()
[HF,HM] = histClassifier(data,data['Sex'],Bx,By,xmin,xmax)



Small Index 167


In [8]:
HM

array([[ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  6,  1,  0,  0],
       [ 0,  1,  0,  2,  7,  8,  3,  0],
       [ 0,  0,  0,  5, 14,  7,  5,  0],
       [ 0,  0,  0,  1,  3,  1, 12,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  1]], dtype=int32)

In [9]:
ht = 69
hs = 17.5
heightBin = (np.round((Bx-1)*(ht - xmin['Height'])/(xmax['Height'] - xmin['Height']))).astype('int32')
headSpanBin = (np.round((By-1)*(hs - xmin['HandSpan'])/(xmax['HandSpan'] - xmin['HandSpan']))).astype('int32')
print(heightBin)
print(headSpanBin)
print(HF[heightBin][headSpanBin])
print(HF[heightBin][headSpanBin] + HM[heightBin][headSpanBin])

4
1
0
1


In [10]:
HF[heightBin][headSpanBin]/(HF[heightBin][headSpanBin] + HM[heightBin][headSpanBin])

0.0

In [11]:
#count of male and female
countFemale = data[data['Sex'] == 'Female']['Height'].count()
print(countFemale)
countMale = data[data['Sex'] == 'Male']['Height'].count()
print(countMale)

89
78


In [12]:
#PDF functions
meanFemale = data[['Height','HandSpan']][data['Sex'] == 'Female'].mean()
meanFemale

Height      65.280899
HandSpan    19.601124
dtype: float64

In [13]:
meanMale = data[['Height','HandSpan']][data['Sex'] == 'Male'].mean()
meanMale

Height      71.294872
HandSpan    22.301282
dtype: float64

In [14]:
CoFemale = data[['Height','HandSpan']][data['Sex'] == 'Female'].cov()
CoFemale

,Height,HandSpan
Height,7.727017,1.647408
HandSpan,1.647408,1.756703


In [15]:
CoMale = data[['Height','HandSpan']][data['Sex'] == 'Male'].cov()
CoMale

,Height,HandSpan
Height,7.041792,1.793124
HandSpan,1.793124,2.060648


In [16]:
type(CoFemale)

pandas.core.frame.DataFrame

In [17]:
a = CoFemale.as_matrix()
Femaledet = np.linalg.det(a)
Femaledet

10.86012331402638

In [18]:
m = CoMale.as_matrix()
Maledet = np.linalg.det(m)
Maledet

11.295359402502296

In [19]:
import math

In [20]:
import math
def pdf(mean, coMatrix, coDet, x):
    var = float(coDet)**0.5
    denom = (2*math.pi*var)**.5
    xsubmean = np.matrix(x-mean)
    print("XsubMean" , xsubmean)
    print("XsubMean" , xsubmean.shape)
    invcovariance = np.linalg.inv(coMatrix)
    print("CobvarianceInverse",invcovariance)
    print("CobvarianceInverse",invcovariance.shape)
    meanInvProduct = xsubmean* invcovariance
    print("meanInvProduct",meanInvProduct.shape)
    print("meanInvProduct",meanInvProduct)
    xsubMeanTranspose= np.transpose([np.array(x-mean)])
    print("xsubMeanTranspose",xsubMeanTranspose)
    print("shape",xsubMeanTranspose.shape)
    num = math.exp(-1 *meanInvProduct *xsubMeanTranspose /2 )
    return num/denom

In [21]:
x = 69
y = 17.5
pdfFemale = pdf(meanFemale,a,Femaledet,[x,y])
pdfMale = pdf(meanMale,m,Maledet,[x,y])
ProbFemale = countFemale * pdfFemale / (countMale * pdfMale + countFemale * pdfFemale  )
print("Probabilty of females ",ProbFemale)

XsubMean [[ 3.71910112 -2.1011236 ]]
XsubMean (1, 2)
CobvarianceInverse [[ 0.16175721 -0.15169331]
 [-0.15169331  0.71150365]]
CobvarianceInverse (2, 2)
meanInvProduct (1, 2)
meanInvProduct [[ 0.92031782 -2.05911987]]
xsubMeanTranspose [[ 3.71910112]
 [-2.1011236 ]]
shape (2, 1)
XsubMean [[-2.29487179 -4.80128205]]
XsubMean (1, 2)
CobvarianceInverse [[ 0.18243312 -0.15874869]
 [-0.15874869  0.62342342]]
CobvarianceInverse (2, 2)
meanInvProduct (1, 2)
meanInvProduct [[ 0.34353662 -2.62892376]]
xsubMeanTranspose [[-2.29487179]
 [-4.80128205]]
shape (2, 1)
Probabilty of females  0.898808650526
